In [3]:
import pygame
import sys
import random

class CyberpunkShooter:
    def __init__(self):
        pygame.init()

        # Constants
        self.WIDTH, self.HEIGHT = 800, 600
        self.PLAYER_SIZE = 50
        self.ENEMY_SIZE = 50
        self.BULLET_SIZE = 5
        self.PLAYER_SPEED = 5
        self.ENEMY_SPEED = 3
        self.BULLET_SPEED = 7
        self.ENEMY_SPAWN_RATE = 5
        self.UPGRADE_SCORE_THRESHOLD = 50
        self.POWERUP_SPAWN_RATE = 2  # Adjust as needed
        self.BOSS_SPAWN_RATE = 60  # Boss spawns every 60 seconds

        # Colors
        self.WHITE = (255, 255, 255)
        self.RED = (255, 0, 0)
        self.GREEN = (0, 255, 0)
        self.BLUE = (0, 0, 255)

        # Create the screen
        self.screen = pygame.display.set_mode((self.WIDTH, self.HEIGHT))
        pygame.display.set_caption("Cyberpunk Shooter")

        # Load images and sounds
        self.player_image = pygame.Surface((self.PLAYER_SIZE, self.PLAYER_SIZE))
        self.player_image.fill(self.WHITE)

        self.enemy_image = pygame.Surface((self.ENEMY_SIZE, self.ENEMY_SIZE))
        self.enemy_image.fill(self.RED)

        self.upgrade_image = pygame.Surface((self.BULLET_SIZE, self.BULLET_SIZE))
        self.upgrade_image.fill(self.BLUE)

        # Initialize bullets as a list of dictionaries
        self.bullets = []

        # Auto-shoot toggle
        self.auto_shoot = False

        # Fonts
        self.font = pygame.font.Font(None, 36)

    def game_over(self, score):
        # Display game over screen
        self.screen.fill((0, 0, 0))
        game_over_text = self.font.render("Game Over", True, self.WHITE)
        score_text = self.font.render(f"Final Score: {score}", True, self.WHITE)
        restart_text = self.font.render("Press SPACE to restart", True, self.WHITE)

        self.screen.blit(game_over_text, (self.WIDTH // 2 - 100, self.HEIGHT // 2 - 50))
        self.screen.blit(score_text, (self.WIDTH // 2 - 80, self.HEIGHT // 2))
        self.screen.blit(restart_text, (self.WIDTH // 2 - 150, self.HEIGHT // 2 + 50))

        pygame.display.flip()

        waiting_for_restart = True
        while waiting_for_restart:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()
                elif event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_SPACE:
                        return True  # Restart the game
        return False  # Quit the game

    def spawn_powerup(self):
        powerup_pos = [random.randint(0, self.WIDTH - self.BULLET_SIZE), 0]
        return powerup_pos

    def draw_controls(self):
        controls_text = self.font.render("Controls:", True, self.WHITE)
        move_text = self.font.render("Arrow keys to move", True, self.WHITE)
        shoot_text = self.font.render("Space to shoot", True, self.WHITE)
        auto_shoot_text = self.font.render("A to toggle auto-shoot", True, self.WHITE)

        self.screen.blit(controls_text, (self.WIDTH - 200, 10))
        self.screen.blit(move_text, (self.WIDTH - 200, 50))
        self.screen.blit(shoot_text, (self.WIDTH - 200, 90))
        self.screen.blit(auto_shoot_text, (self.WIDTH - 200, 130))

    def draw_boss(self, boss):
        pygame.draw.rect(self.screen, boss["color"], (boss["position"][0], boss["position"][1], boss["size"], boss["size"]))
        pygame.draw.rect(self.screen, self.WHITE, (boss["position"][0], boss["position"][1] - 10, boss["size"], 5))  # Boss health bar

    def run(self):
        player_pos = [self.WIDTH // 2, self.HEIGHT // 2]
        enemies = []
        powerups = []
        boss = None
        clock = pygame.time.Clock()
        score = 0
        health = 100
        level = 1
        bullet_upgrade_level = 1  # Number of bullets shot at a time
        rainbow_bullet_counter = 0
        auto_shoot_counter = 0
        boss_spawn_timer = 0

        pygame.time.set_timer(pygame.USEREVENT, 1000)  # Timer event every second

        while True:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()
                elif event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_SPACE:
                        # Shoot rainbow bullets when rainbow_bullet_counter reaches 15
                        if rainbow_bullet_counter >= 15:
                            self.bullets.append({"position": [player_pos[0] + self.PLAYER_SIZE // 2, player_pos[1]], "color": (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))})
                            rainbow_bullet_counter = 0
                        else:
                            self.bullets.append({"position": [player_pos[0] + self.PLAYER_SIZE // 2, player_pos[1]], "color": self.GREEN})
                    elif event.key == pygame.K_a:
                        self.auto_shoot = not self.auto_shoot  # Toggle auto-shoot

            keys = pygame.key.get_pressed()
            if keys[pygame.K_LEFT] and player_pos[0] > 0:
                player_pos[0] -= self.PLAYER_SPEED
            if keys[pygame.K_RIGHT] and player_pos[0] < self.WIDTH - self.PLAYER_SIZE:
                player_pos[0] += self.PLAYER_SPEED
            if keys[pygame.K_UP] and player_pos[1] > 0:
                player_pos[1] -= self.PLAYER_SPEED
            if keys[pygame.K_DOWN] and player_pos[1] < self.HEIGHT - self.PLAYER_SIZE:
                player_pos[1] += self.PLAYER_SPEED

            # Auto-shoot
            if self.auto_shoot and auto_shoot_counter >= 10:
                self.bullets.append({"position": [player_pos[0] + self.PLAYER_SIZE // 2, player_pos[1]], "color": (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))})
                auto_shoot_counter = 0
            auto_shoot_counter += 1

            # Update rainbow_bullet_counter
            rainbow_bullet_counter += 1

            # Spawn boss after every 60 seconds
            if boss_spawn_timer >= self.BOSS_SPAWN_RATE and boss is None:
                boss = {
                    "position": [random.randint(0, self.WIDTH - self.ENEMY_SIZE), 0],
                    "color": (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)),
                    "size": random.randint(30, 80),
                    "health": random.randint(50, 200),
                }
                boss_spawn_timer = 0

            # Move boss
            if boss:
                boss["position"][1] += (self.ENEMY_SPEED + level * 0.1)

            # Move enemies
            for enemy in enemies:
                enemy[1] += self.ENEMY_SPEED

            # Move bullets
            for bullet in self.bullets:
                bullet["position"][1] -= self.BULLET_SPEED

            # Collision detection: Bullet hits boss
            if boss:
                for bullet in self.bullets:
                    if (
                        bullet["position"][0] < boss["position"][0] + boss["size"]
                        and bullet["position"][0] + self.BULLET_SIZE > boss["position"][0]
                        and bullet["position"][1] < boss["position"][1] + boss["size"]
                        and bullet["position"][1] + self.BULLET_SIZE > boss["position"][1]
                    ):
                        boss["health"] -= bullet_upgrade_level * 10
                        self.bullets.remove(bullet)
                        if boss["health"] <= 0:
                            score += 100
                            boss = None

            # Collision detection: Bullet hits enemy
            for bullet in self.bullets:
                for enemy in enemies:
                    if (
                        bullet["position"][0] < enemy[0] + self.ENEMY_SIZE
                        and bullet["position"][0] + self.BULLET_SIZE > enemy[0]
                        and bullet["position"][1] < enemy[1] + self.ENEMY_SIZE
                        and bullet["position"][1] + self.BULLET_SIZE > enemy[1]
                    ):
                        enemies.remove(enemy)
                        score += 10
                        self.bullets.remove(bullet)

            # Collision detection: Enemy hits player
            for enemy in enemies:
                if (
                    player_pos[0] < enemy[0] + self.ENEMY_SIZE
                    and player_pos[0] + self.PLAYER_SIZE > enemy[0]
                    and player_pos[1] < enemy[1] + self.ENEMY_SIZE
                    and player_pos[1] + self.PLAYER_SIZE > enemy[1]
                ):
                    enemies.remove(enemy)
                    health -= 10
                    if health <= 0:
                        restart = self.game_over(score)
                        if not restart:
                            return

            # Collision detection: Bullet hits power-up
            for bullet in self.bullets:
                for powerup in powerups:
                    if (
                        bullet["position"][0] < powerup[0] + self.BULLET_SIZE
                        and bullet["position"][0] + self.BULLET_SIZE > powerup[0]
                        and bullet["position"][1] < powerup[1] + self.BULLET_SIZE
                        and bullet["position"][1] + self.BULLET_SIZE > powerup[1]
                    ):
                        powerups.remove(powerup)
                        bullet_upgrade_level += 1
                        print(f"Bullet Upgrade: {bullet_upgrade_level}")

            # Spawn power-ups randomly
            if random.randint(1, 100) < self.POWERUP_SPAWN_RATE:
                powerup_pos = self.spawn_powerup()
                powerups.append(powerup_pos)

            # Move power-ups
            for powerup in powerups:
                powerup[1] += (self.ENEMY_SPEED + level * 0.1)

            # Remove off-screen enemies, bullets, and power-ups
            enemies = [enemy for enemy in enemies if enemy[1] < self.HEIGHT]
            self.bullets = [bullet for bullet in self.bullets if bullet["position"][1] > 0]
            powerups = [powerup for powerup in powerups if powerup[1] < self.HEIGHT]

            # Check for level completion
            if score >= level * 100:
                level += 1
                print(f"Level {level} Complete!")
                health = 100

            # Draw everything
            self.screen.fill((0, 0, 0))
            self.screen.blit(self.player_image, player_pos)
            if boss:
                self.draw_boss(boss)
            else:
                for enemy in enemies:
                    self.screen.blit(self.enemy_image, enemy)
            for bullet in self.bullets:
                pygame.draw.rect(self.screen, bullet["color"], (bullet["position"][0], bullet["position"][1], self.BULLET_SIZE, self.BULLET_SIZE))
            for powerup in powerups:
                self.screen.blit(self.upgrade_image, powerup)

            # Display score, health, and level
            score_text = self.font.render(f"Score: {score}", True, self.WHITE)
            health_text = self.font.render(f"Health: {health}", True, self.WHITE)
            level_text = self.font.render(f"Level: {level}", True, self.WHITE)
            upgrade_text = self.font.render(f"Bullet Upgrade: {bullet_upgrade_level}", True, self.WHITE)
            auto_shoot_text = self.font.render(f"Auto Shoot: {'On' if self.auto_shoot else 'Off'}", True, self.WHITE)
            self.screen.blit(score_text, (10, 10))
            self.screen.blit(health_text, (10, 50))
            self.screen.blit(level_text, (10, 90))
            self.screen.blit(upgrade_text, (10, 130))
            self.screen.blit(auto_shoot_text, (10, 170))

            # Draw controls
            self.draw_controls()

            # Update the display
            pygame.display.flip()

            # Cap the frame rate
            clock.tick(60)
            boss_spawn_timer += 1

# Run the game
game_instance = CyberpunkShooter()
game_instance.run()


SystemExit: 